# Project: RADET Preprocessing

## Introduction

This script is for precessing the RADET file for data visualization.

## Import Libraries

In [3]:
import numpy as np
import pandas as pd
import shutil, os
from glob import glob
import pyarrow as pq
from datetime import datetime
import matplotlib.pyplot as plt
from datetime import timedelta, date
pd.set_option('display.max.rows', None)
pd.set_option('display.max.columns', None)
pd.set_option('display.width', None)

### Enter Radet File Name to be processed

**Note:** Put the script and the radet file in the same folder!

In [1]:
radet_file_name = input(str())

ACE 1 Combined RADET 05-12-2022


### Create Source Folder

In [4]:
#Old Radet Source files
source = 'C:/Users/TIMOTHY/Desktop/RADET/'

#Create output directory for combined radet
out_dir = "Power BI"

if os.path.exists(source + out_dir):
    pass
else:
    output_path = os.path.join(source, out_dir)
    os.mkdir(output_path)

## Import Dataset

In [5]:
# import dataset into pandas dataframe
df1 = pd.read_excel(radet_file_name + '.xlsx')

In [6]:
# Make a copy of the dataframe
df = df1.copy()

# preview sample of observation
df.sample(2)

,State,LGA,Facility,Patient ID,Hospital Number,Unique ID,Household Unique No,Received OVC Service?,Sex,Current Weight (Kg),Pregnancy Status,Date of Birth (yyyy-mm-dd),ART Start Date (yyyy-mm-dd),Last Pickup Date (yyyy-mm-dd),Months of ARV Refill,Regimen Line at ART Start,Regimen at ART Start,Date of Start of current ART Regimen (yyyy-mm-dd),Current Regimen Line,Current ART Regimen,Clinical Staging at Last Visit,Date of Last CD4 Count,Last CD4 Count,Date of last TB-LAM,Last TB-LAM,Date of last Cryptococal Antigen (yyyy-mm-dd),Last Cryptococal Antigen,Date of Full Disclosure (yyyy-mm-dd),Date Enrolled on OTZ (yyyy-mm-dd),Number of Support Group (OTZ Club) meeting attended,Number of OTZ Modules completed,Date of Viral Load Sample Collection (yyyy-mm-dd),Current Viral Load (c/ml),Date of Current Viral Load (yyyy-mm-dd),Viral Load Indication,VL Result After VL Sample Collection (c/ml),Date of VL Result After VL Sample Collection (yyyy-mm-dd),Status at Registration,Date of Enrollment/Transfer-In (yyyy-mm-dd),ART Enrollment Setting,Previous ART Status,Confirmed Date of Previous ART Status (yyyy-mm-dd),Current ART Status,Date of Current ART Status (yyyy-mm-dd),RTT,"If Dead, Cause of Dead",VA Cause of Dead,"If Transferred out, new facility",Reason for Transfer-Out / IIT / Stooped Treatment,TB status at Last Visit,Date of TB Screening (yyyy-mm-dd),TB Screening Outcome,Date of TB Sample Collection (yyyy-mm-dd),TB Sample Test Type,Date of TB Sample Result Received (yyyy-mm-dd),TB Sample Result,Date of Start of TB Treatment (yyyy-mm-dd),"TB Treatment Type (new, relapsed etc)",Date of Completion of TB Treatment (yyyy-mm-dd),TB Treatment Outcome,Date of TPT Start (yyyy-mm-dd),TPT Type,TPT Completion date (yyyy-mm-dd),TPT Completion status,Date of Commencement of EAC (yyyy-mm-dd),Number of EAC Sessions Completed,Date of last EAC Session Completion (yyyy-mm-dd),Date of Extended EAC Completion (yyyy-mm-dd),Date of Repeat Viral Load - Post EAC VL Sample Collected (yyyy-mm-dd),Repeat Viral load result (c/ml)- POST EAC,Date of Repeat Viral load result - POST EAC VL (yyyy-mm-dd),Date Commenced DSD Model (yyyy-mm-dd),DSD Model,Date of Return of DSD Client to Facility (yyyy-mm-dd),Screening for Chronic Conditions,Co-morbidities,Date of Cervical Cancer Screening (yyyy-mm-dd),Cervical Cancer Screening Type,Cervical Cancer Screening Method,Result of Cervical Cancer Screening,Date of Precancerous Lesions Treatment (yyyy-mm-dd),Precancerous Lesions Treatment Methods,Date Biometrics Enrolled (yyyy-mm-dd),Valid Biometrics Enrolled?,Number of Fingers Captured,IIT Chance (%),Date calculated (yyyy-mm-dd),Case Manager
27559,Borno,Jere,bo University of Maiduguri Teaching Hospital,90cea043-4263-42a3-b2fe-0c84a17136b5,UM-07-3039,NaN,NaN,NaN,Female,67.0,Not pregnant,1982-02-25,2007-12-05,2007-12-05,1.0,Adult.1st.Line,AZT-3TC-NVP,2007-12-05 00:00:00,Adult.1st.Line,AZT-3TC-NVP,NaN,2011-04-04 00:00:00,525,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,2011-04-04,200,2011-04-04,Routine - Routine,200,2011-04-04,HIV+ non ART,2007-12-05 00:00:00,NaN,Active,2007-12-05,IIT,2008-02-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN
12879,Borno,Monguno,bo Monguno General Hospital,47f9875a-2854-48e1-99bf-09af83275784,GHM-20-0559,GHM-20-0559,NaN,NaN,Female,NaN,Not pregnant,2000-03-30,2020-03-30,2022-08-15,3.0,Adult.1st.Line,TDF-3TC-DTG,2020-03-30 00:00:00,Adult.1st.Line,TDF-3TC-DTG,Stage I,NaN,NaN,NaT,NaN,NaT,NaN,NaT,NaT,NaN,NaN,2022-08-15,229,2022-09-16,Routine - Routine,229,2022-09-16,HIV+ non ART,2020-03-30 00:00:00,Facility,IIT,2021-07-28,Active,2022-08-15 00:00:00,NaN,NaN,NaN,NaN,NaN,No sign or symptoms of TB,2022-08-15,No TB sign (not a suspect),NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,2020-03-30,Isoniazid 300mg,2020-09-26,NaN,NaT,NaN,NaT,NaT,NaT,NaN,NaT,NaT,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,SANUSI USMAN


In [7]:
# Shape of the dataset
df.shape

(114250, 88)

### Functions

In [8]:
# Change date to datetime
def renameDate (df):
    dates = df.columns[df.columns.str.contains('Date') | df.columns.str.contains('date') ].to_list()
    for date in dates:
        df[date] = pd.to_datetime(df[date], errors='coerce')
    return df

# Generate next appointment date
def nextAppointmentDate(df, pickupDate, refillMonth):
    df['Next Appointment Date'] =df['Last Pickup Date (yyyy-mm-dd)'] +\
    pd.to_timedelta(df['Months of ARV Refill'] * 30,unit='d')
    
    return df

# Generate IIT Date
def iitDate(df, lastPickupDate, nextAppointmentDate, monthOfRefill):
    df[nextAppointmentDate] = df[lastPickupDate] +\
    pd.to_timedelta(df[monthOfRefill] * 30, unit='d')
    
    df['IIT Date'] = df[nextAppointmentDate] + timedelta(days=29)
    
    return df

# Generate True Current ART Status
def trueCurrentARTStatus(df, IIT_Date, date):
    # Create Active and Inactive column
    #today = date.today()
    today = pd.Timestamp(date)
    today = today.strftime('%Y-%m-%d')
    df.loc[df['IIT Date'] > today, 'Active/Inactive'] = 'Active'
    df.loc[df['IIT Date'] <= today, 'Active/Inactive'] = 'Inactive'
    
    # Create True Current ART Status column
    df.loc[(df['Current ART Status'] == 'Active') & (df['Active/Inactive'] == 'Active') , 'True Current ART Status'] = 'Active'
    df.loc[(df['Current ART Status'] == 'Active') & (df['Active/Inactive'] == 'Inactive') , 'True Current ART Status'] = 'IIT'
    df.loc[(df['Current ART Status'] == 'Active-Restart') & (df['Active/Inactive'] == 'Active') , 'True Current ART Status'] = 'Active-Restart'
    df.loc[(df['Current ART Status'] == 'Active-Restart') & (df['Active/Inactive'] == 'Inactive') , 'True Current ART Status'] = 'IIT'
    df.loc[(df['Current ART Status'] == 'Active-Transfer In') & (df['Active/Inactive'] == 'Active') , 'True Current ART Status'] = 'Active-Transfer In'
    df.loc[(df['Current ART Status'] == 'Active-Transfer In') & (df['Active/Inactive'] == 'Inactive') , 'True Current ART Status'] = 'IIT'
    df.loc[(df['Current ART Status'] == 'Active-RTC') & (df['Active/Inactive'] == 'Active') , 'True Current ART Status'] = 'Active-RTC'
    df.loc[(df['Current ART Status'] == 'Active-RTC') & (df['Active/Inactive'] == 'Inactive') , 'True Current ART Status'] = 'IIT'
    df.loc[(df['Current ART Status'] == 'IIT') & (df['Active/Inactive'] == 'Active') , 'True Current ART Status'] = 'Active'
    df.loc[(df['Current ART Status'] == 'IIT') & (df['Active/Inactive'] == 'Inactive') , 'True Current ART Status'] = 'IIT'
    df.loc[(df['Current ART Status'] == 'Transferred Out') , 'True Current ART Status'] = 'Transferred Out'
    df.loc[(df['Current ART Status'] == 'Dead') , 'True Current ART Status'] = 'Dead'
    df.loc[(df['Current ART Status'] == 'Stopped') , 'True Current ART Status'] = 'Stopped'
    return df

# Viral Load Expiry Date
def vlExpiryDate(df, ARTduration):
    # Condition 1
    df.loc[(df['True Current ART Status'] == 'Active') & (df[ARTduration] > 90) \
           & (df['Date of Current Viral Load (yyyy-mm-dd)'] != pd.NaT), 'VL Expiry Date'] = df['Date of Current Viral Load (yyyy-mm-dd)'] + timedelta(365)

    # Condition 2
    df.loc[(df['True Current ART Status'] == 'Active') & (df[ARTduration] > 90) \
           & (df['Date of Current Viral Load (yyyy-mm-dd)'] == pd.NaT), 'VL Expiry Date'] = pd.Timestamp('now')

    # Condition 3
    df.loc[(df['True Current ART Status'].isin(['Active-Restart', 'Active-Transfer In'])) & (df[ARTduration] > 90) \
           & (df['Date of Current Viral Load (yyyy-mm-dd)'] != pd.NaT), 'VL Expiry Date'] = df['Date of Current Viral Load (yyyy-mm-dd)'] + timedelta(365)

    # Condition 4
    df.loc[(df['True Current ART Status'].isin(['Active-Restart', 'Active-Transfer In'])) & (df[ARTduration] > 90) \
           & (df['Date of Current Viral Load (yyyy-mm-dd)'] == pd.NaT), 'VL Expiry Date'] = pd.Timestamp('now')
    
    return df

# Viral Load Status
def viralLoadStatus(df):
    # Replace Invalid strings in observation and convert to float
    df['Current Viral Load (c/ml)'].replace({'107276`': '107276', '<20': '19'}, inplace=True)
    df['Current Viral Load (c/ml)'] = df['Current Viral Load (c/ml)'].astype('float')
    
    df.loc[df['Current Viral Load (c/ml)'] < 51, 'Viral Load Category'] = 'Undetectable'
    df.loc[df['Current Viral Load (c/ml)'].between(51, 999), 'Viral Load Category'] = 'LLV'
    df.loc[df['Current Viral Load (c/ml)'] > 999, 'Viral Load Category'] = 'Unsuppressed'
    
    df.loc[df['Current Viral Load (c/ml)'] >= 1000, 'Viral Load Status'] = 'Unsuppressed'
    df.loc[df['Current Viral Load (c/ml)'] < 1000, 'Viral Load Status'] = 'Suppressed'
    
    return df

# Calculate Age Function
def calculateAge(df, dob):
    now = pd.Timestamp('now')
    df[dob] = pd.to_datetime(df[dob], format= '%m%d%y')
    df[dob] = df[dob].where(df[dob] < now, df[dob] - np.timedelta64(100, 'Y'))
    df['Age'] = (now - df[dob]).astype('<m8[Y]').astype(int)
    
    return df

# Generate Age Range
def ageRange(df):
    df.loc[df['Age'] < 1, 'Age Range'] = '<1'
    df.loc[df['Age'].between(1, 4), 'Age Range'] = '1-4'
    df.loc[df['Age'].between(5, 9), 'Age Range'] = '5-9'
    df.loc[df['Age'].between(10, 14), 'Age Range'] = '10-14'
    df.loc[df['Age'].between(15, 19), 'Age Range'] = '15-19'
    df.loc[df['Age'].between(20, 24), 'Age Range'] = '20-24'
    df.loc[df['Age'].between(25, 29), 'Age Range'] = '25-29'
    df.loc[df['Age'].between(30, 34), 'Age Range'] = '30-34'
    df.loc[df['Age'].between(35, 39), 'Age Range'] = '35-39'
    df.loc[df['Age'].between(40, 44), 'Age Range'] = '40-44'
    df.loc[df['Age'].between(45, 49), 'Age Range'] = '45-49'
    df.loc[df['Age'].between(50, 54), 'Age Range'] = '50-54'
    df.loc[df['Age'].between(55, 59), 'Age Range'] = '55-59'
    df.loc[df['Age'].between(60, 64), 'Age Range'] = '60-64'
    df.loc[df['Age'] > 64, 'Age Range'] = '65+'
    
    df.loc[df['Age Range'] == '<1', 'Age Band No'] = 1
    df.loc[df['Age Range'] == '1-4', 'Age Band No'] = 2
    df.loc[df['Age Range'] == '5-9', 'Age Band No'] = 3
    df.loc[df['Age Range'] == '10-14', 'Age Band No'] = 4
    df.loc[df['Age Range'] == '15-19', 'Age Band No'] = 5
    df.loc[df['Age Range'] == '20-24', 'Age Band No'] = 6
    df.loc[df['Age Range'] == '25-29', 'Age Band No'] = 7
    df.loc[df['Age Range'] == '30-34', 'Age Band No'] = 8
    df.loc[df['Age Range'] == '35-39', 'Age Band No'] = 9
    df.loc[df['Age Range'] == '40-44', 'Age Band No'] = 10
    df.loc[df['Age Range'] == '45-49', 'Age Band No'] = 11
    df.loc[df['Age Range'] == '50-54', 'Age Band No'] = 12
    df.loc[df['Age Range'] == '55-59', 'Age Band No'] = 13
    df.loc[df['Age Range'] == '60-64', 'Age Band No'] = 14
    df.loc[df['Age Range'] == '65+', 'Age Band No'] = 15
    
    # Change Age Band Number to Integer
    df['Age Band No'] = df['Age Band No'].astype('int')
    
    return df

# Generate Age range OTZ
def ageRangeOTZ(df):
    df.loc[df['Age'].between(0, 9), 'Age Range OTZ'] = 'others'
    df.loc[df['Age'].between(10, 24), 'Age Range OTZ'] = '10-24'
    df.loc[df['Age'] > 24, 'Age Range OTZ'] = 'others'
    
    return df

# Calculate Duration
def calculateDuration(df, start_date, end_date, time_unit, column_name):
    
    if time_unit == 'day':
        df[column_name] = (pd.Timestamp(end_date)-df[start_date]).astype('<m8[D]').fillna(0).astype('int')
        return df
    elif time_unit == 'month':
        df[column_name] = (pd.Timestamp(end_date)-df[start_date]).astype('<m8[M]').fillna(0).astype('int')
        return df
    elif time_unit == 'year':
        df[column_name] = (pd.Timestamp(end_date)-df[start_date]).astype('<m8[Y]').fillna(0).astype('int')
        return df

# Generate Fiscal Year column
def fiscalYear(df, date):
    # Create Fiscal Quarter to extract Fiscal Year
    df['As Quarter'] = df[date].dt.to_period('Q-SEP')
   
    # Convert Fiscal Quarter to String
    df['As Quarter'] = df['As Quarter'].fillna(0).astype('str')
    
    # Select Year(i.e 22, 23 e.t.c) from Fiscal Quarter
    df['Dummy Year'] = df['As Quarter'].str[2:4]
    
    # Select Quarter(i.e Q1, Q2, Q3, Q4) from Fiscal Quarter
    df['Fiscal Quarter'] = df['As Quarter'].str[-2:]
    
    # Create Fiscal Year Column
    df['Fiscal Year'] = df['Dummy Year'] + '-' + df['Fiscal Quarter']
    
    # Remove redundant columns
    df.drop(['Dummy Year', 'As Quarter'], axis=1, inplace=True)
    
    # Create Quarter Number using Fiscal Quarter
    df.loc[df['Fiscal Quarter'] == 'Q1', 'Fiscal Quarter No'] = 1
    df.loc[df['Fiscal Quarter'] == 'Q2', 'Fiscal Quarter No'] = 2
    df.loc[df['Fiscal Quarter'] == 'Q3', 'Fiscal Quarter No'] = 3
    df.loc[df['Fiscal Quarter'] == 'Q4', 'Fiscal Quarter No'] = 4
    
    # Convert Fiscal Quarter Number to Integer from Floating point.
    df['Fiscal Quarter No'].fillna(np.nan, inplace=True)
    
    
    return df

# Generate Imminent IIT column
def imminentIIT(df, nextappointmentdate, currentARTstatus):
    # Condition 1
    df.loc[(pd.Timestamp('now') < (df[nextappointmentdate])) & \
       (df[currentARTstatus].isin(['Active', 'Active-Restart', 'Active-Transfer In', 'Active-RTC'])), 'Imminent IIT'] = 'Active'
    
    # Condition 2
    df.loc[(pd.Timestamp('now') > df[nextappointmentdate]) & \
    ((pd.Timestamp('now') - df[nextappointmentdate]) < pd.Timedelta(29, 'd')) & \
       (df[currentARTstatus].isin(['Active', 'Active-Restart', 'Active-Transfer In', 'Active-RTC'])), 'Imminent IIT'] = 'Imminent IIT'
    
    # Condition 3
    df.loc[(pd.Timestamp('now') > df[nextappointmentdate]) & \
    ((pd.Timestamp('now') - df[nextappointmentdate]) >= pd.Timedelta(29, 'd')) & \
       (df[currentARTstatus].isin(['Active', 'Active-Restart', 'Active-Transfer In', 'Active-RTC'])), 'Imminent IIT'] = 'IIT'
    
    return df

### Data Preprocessing

In [9]:
df = renameDate(df)

In [10]:
df = nextAppointmentDate(df, 'Last Pickup Date (yyyy-mm-dd)', 'Months of ARV Refill')

In [11]:
df = iitDate(df, 'Last Pickup Date (yyyy-mm-dd)', 'Next Appointment Date', 'Months of ARV Refill')

In [12]:
df = trueCurrentARTStatus(df, 'IIT Date', '2022-12-05')

In [13]:
df = calculateDuration(df, 'ART Start Date (yyyy-mm-dd)', '2022-12-05', 'day', 'ART Duration[Days]')

In [14]:
df = vlExpiryDate(df, 'ART Duration[Days]')

In [15]:
df = viralLoadStatus(df)

In [16]:
df = calculateAge(df, 'Date of Birth (yyyy-mm-dd)')

In [17]:
df = ageRange(df)

In [18]:
df = ageRangeOTZ(df)

In [19]:
df = fiscalYear(df, 'Next Appointment Date')

In [20]:
df = calculateDuration(df, 'Date of Current Viral Load (yyyy-mm-dd)', 'now', 'day', 'VL Duration [Days]')

In [21]:
df = imminentIIT(df, 'Next Appointment Date', 'Current ART Status')

### Create Processed and Destination File Directory and save file

In [22]:
#  Create Processed Files directory
processed_dir = "Processed Files"

if os.path.exists(source + processed_dir):
    pass
else:
    output_path = os.path.join(source, processed_dir)
    os.mkdir(output_path)

In [23]:
destination = 'C:/Users/TIMOTHY/Desktop/RADET/Power BI/RADET.xlsx'
processed_file_destination = 'C:/Users/TIMOTHY/Desktop/RADET/Processed Files'

if os.path.exists(destination):
    # Instant Time
    now = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    file_name = "RADET " + now
    # Move desc file to another folder
    shutil.move(destination, processed_file_destination)
    # Rename moved file
    old_name = processed_file_destination + '/RADET.xlsx'
    new_name = f'C:/Users/TIMOTHY/Desktop/RADET/Processed Files/{file_name}.xlsx'
    os.rename(old_name, new_name)
    # Save new file to directory
    df.to_excel(destination, index=False)
else:
    df.to_excel(destination, index=False)

### Change Processed File to Parquet

In [ ]:
# Change Processed excel files to Parquet
for f in glob(processed_file_destination + "/*.xlsx"):
    file = pd.read_excel(f)
    file.to_parquet(processed_file_destination + f"/{f[-30:-5]}.parquet")

### Delete the Excel Files already converted to Parquet

In [ ]:
# Delete Excel Files
for f in glob(processed_file_destination + "/*.xlsx"):
    os.remove(f)